In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../Weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt

In [ ]:
# Reloadable libraries
import finite_diff, importlib
importlib.reload(finite_diff)
from finite_diff import fd_hessian_test, fd_gradient_test, gradient_convergence_plot, hessian_convergence_plot, hessian_convergence_block_plot

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir, 'models/equal_edge_atI.obj')
SUBDIVISION_RESOLUTION = 5

In [ ]:
with so():
    dodecahedron_linkage = elastic_rods.RodLinkage(MODEL_PATH, SUBDIVISION_RESOLUTION, False)
    driver=dodecahedron_linkage.centralJoint()
    dodecahedron_linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
#     restlen_solve is broken until the design parameter refactoring is done.
    # elastic_rods.restlen_solve(dodecahedron_linkage)
    jdo = dodecahedron_linkage.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    elastic_rods.compute_equilibrium(dodecahedron_linkage, fixedVars=fixedVars)
    dodecahedron_view = linkage_vis.LinkageViewer(dodecahedron_linkage, width=1024, height=640)
    dodecahedron_view.setCameraParams(default_camera_parameters)
# dodecahedron_view.show()

strips = order_segments_by_ribbons(dodecahedron_linkage)

In [ ]:
# test rod 1
test_rod, fixedVars = construct_elastic_rod_loop_from_rod_segments(dodecahedron_linkage, strips[0])
# Set the material of the new rod to be the same as previously.
test_rod.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))

In [ ]:
# # test rod 2
# centerline_pos = np.array([[0, 0, 0], [1, 0, 0], [2.1, 0, 0], [3, 0, 0]])
# deformed_centerline_pos = np.array([[0, 0, 0],[1, 0, 0],[1 + 0.6 * np.sqrt(0.5),  0.6 * np.sqrt(0.5), 0],[2 + 0.6 * np.sqrt(0.5),  0.6 * np.sqrt(0.5), 0]])
# test_rod = elastic_rods.ElasticRod(centerline_pos)
# test_rod.setDeformedConfiguration(deformed_centerline_pos, np.array([0, np.pi / 6.0, np.pi / 6.0]))
# test_rod.setMaterial(elastic_rods.RodMaterial('ellipse', 200, 0.3, (0.1, 0.05), stiffAxis = elastic_rods.StiffAxis.D1))

In [ ]:
# The Hessian formulas are only accurate with an updated source frame.
test_rod.updateSourceFrame()

In [ ]:
variableDP = True
grad = test_rod.gradEnergyBend(variableDesignParameters = variableDP)

In [ ]:
test_rod.bendingEnergyType = elastic_rods.BendingEnergyType.Bergou2010

#### Perturb the rod

In [ ]:
test_rod.setExtendedDoFs(test_rod.getExtendedDoFs() + np.random.uniform(-5, 5, test_rod.numExtendedDoF()))
test_rod.updateSourceFrame()

### Test Elastic Rod Hessian Vector Product

In [ ]:
# Get Hessian
variable_DP = True
hessian = test_rod.hessian(elastic_rods.EnergyType.Full, variable_DP)
hessian.reflectUpperTriangle()
hessian = hessian.compressedColumn()

In [ ]:
# offsets
theta_offset = test_rod.numDoF() - test_rod.numEdges()
rl_offset = theta_offset + test_rod.numEdges()
rk_offset = rl_offset + test_rod.numEdges()
print(theta_offset, rk_offset, rl_offset)

In [ ]:
var_types = ['FULL', 'POS', 'THETA', 'RL', 'RK']
var_indices = {'FULL': range(0, test_rod.numExtendedDoF()),
                'POS': range(0, theta_offset),
              'THETA': range(theta_offset, rl_offset),
                 'RL': range(rl_offset, rk_offset),
                 'RK': range(rk_offset, test_rod.numExtendedDoF())}

In [ ]:
n_dof = test_rod.numExtendedDoF() if variable_DP else test_rod.numDoF()
print("Num of DOF: ", n_dof)
perturb = np.random.uniform(-1, 1, n_dof)

for i, vi in enumerate(var_types[1:]):
    for j, vj in enumerate(var_types[1:]):
        input_vector = np.zeros_like(perturb)
        input_vector[var_indices[vj]] = perturb[var_indices[vj]]
        code_output = test_rod.applyHessian(input_vector, variable_DP)
        matrix_output = hessian * input_vector
        code_output = code_output[var_indices[vi]]
        matrix_output = matrix_output[var_indices[vi]]
        error = la.norm(code_output - matrix_output) / la.norm(code_output)
        print('Error for {} {} block: {}'.format(vi, vj, error))

### Test Linkage Hessian Vector Product

In [ ]:
# # Single Linkage Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [0.01, 0.01]
# MODEL_PATH = osp.join('/Users/yren/Develop/EPFL_LGG/add_params_elastic_rods/examples/' + 'single_linkage.obj')
# SUBDIVISION_RESOLUTION = 5

In [ ]:
linkage = elastic_rods.RodLinkage(MODEL_PATH, SUBDIVISION_RESOLUTION, False)
driver=linkage.centralJoint()
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))

#### Perturb the linkage

In [ ]:
linkage.setExtendedDoFs(linkage.getExtendedDoFs() + np.random.uniform(-15, 15, linkage.numExtendedDoF()))

linkage.updateSourceFrame()

In [ ]:
# Get Hessian
variable_DP = True
hessian = linkage.hessian(elastic_rods.EnergyType.Full, variable_DP)
hessian.reflectUpperTriangle()
hessian = hessian.compressedColumn()

In [ ]:
n_dof = linkage.numExtendedDoF() if variable_DP else linkage.numDoF()
print("Num of DOF: ", n_dof)
perturb = np.random.uniform(-1, 1, n_dof)
input_vector = perturb
# input_vector[var_indices[vj]] = perturb[var_indices[vj]]
code_output = linkage.applyHessian(input_vector, variable_DP)
matrix_output = hessian * input_vector
# code_output = code_output[var_indices[vi]]
# matrix_output = matrix_output[var_indices[vi]]
error = la.norm(code_output - matrix_output) / la.norm(code_output)
print(error)

#### Per Segment Rest Len Test

In [ ]:
# Get Hessian
PSRL_hessian = linkage.hessianPerSegmentRestlen(elastic_rods.EnergyType.Full)
PSRL_hessian.reflectUpperTriangle()
PSRL_hessian = PSRL_hessian.compressedColumn()

In [ ]:
# offsets
jv_offset = linkage.dofOffsetForJoint(0)
rk_offset = linkage.restKappaDofOffsetForSegment(0)
rl_offset = linkage.restLenDofOffsetForSegment(0)

print(rk_offset, rl_offset)

In [ ]:
var_types = ['FULL', 'RV', 'JV', 'RL', 'RK']
var_indices = {'FULL': range(0, linkage.numExtendedDoFPSRL()),
                'RV': range(0, jv_offset),
                'JV' : range(jv_offset, rk_offset),
                'RK': range(rk_offset, rl_offset),
                'RL': range(rl_offset, linkage.numExtendedDoFPSRL())}

In [ ]:
n_dof = linkage.numExtendedDoFPSRL()
print("Num of DOF: ", n_dof)
perturb = np.random.uniform(-1, 1, n_dof)

for i, vi in enumerate(var_types[1:]):
    for j, vj in enumerate(var_types[1:]):
        input_vector = np.zeros_like(perturb)
        input_vector[var_indices[vj]] = perturb[var_indices[vj]]
        code_output = linkage.applyHessianPerSegmentRestlen(input_vector)
        matrix_output = PSRL_hessian * input_vector
        code_output = code_output[var_indices[vi]]
        matrix_output = matrix_output[var_indices[vi]]
        error = la.norm(code_output - matrix_output) / la.norm(code_output)
        print('Error for {} {} block: {}'.format(vi, vj, error))

In [ ]:
n_dof = linkage.numExtendedDoFPSRL()
print("Num of DOF: ", n_dof)
perturb = np.random.uniform(-1, 1, n_dof)
input_vector = perturb
# input_vector[var_indices[vj]] = perturb[var_indices[vj]]
code_output = linkage.applyHessianPerSegmentRestlen(input_vector)
print(len(code_output))
matrix_output = PSRL_hessian * input_vector
print(len(matrix_output))
# code_output = code_output[var_indices[vi]]
# matrix_output = matrix_output[var_indices[vi]]
error = la.norm(code_output - matrix_output) / la.norm(code_output)
print(error)